# Google Slides Image Uploader:

## A program to help automate the process of batch uploading images into Google Slides presentations

By Kenneth Burchfiel

Released under the MIT license

For more information about this program, please see my corresponding blog post (link forthcoming).

In addition, to see a video of the program in action, visit https://www.youtube.com/watch?v=vzr3K8mqvnE . 


# Background

I created this script in order to help automate the task of uploading images to Google Slides presentations. This script will be particularly helpful if you have a set of images (such as a series of graphs) that need to be updated over time.

In order to insert an image into a Google Slides slide, you need to provide a public URL to that image. This creates a challenge in the event that you don't want the image to be available for anyone to access. This program helps address that issue by generating a signed URL to each image, then deleting that image from Google Cloud Storage (where the images are hosted) soon after it gets inserted into its slide.

If an older copy of the image already exists within a given slide, the script will delete that copy **as long as the new image name is the same as the old one.** If you've changed the image file names, you'll need to manually delete the old images from Google Slides (which shouldn't take long).


# Prerequisites

Before you can apply this code to your own projects, you'll need to perform some setup tasks. To be honest, I didn't keep detailed notes on how I performed these tasks on my end, but I hope these notes will help you get everything set up on your end.

## Step 1:
Open a new Google Cloud Platform project. I used the Google Cloud Console to accomplish this step. For instructions, go to https://cloud.google.com/resource-manager/docs/creating-managing-projects#console. 

NOTE: You may incur expenses when using the Google Cloud Platform.

## Step 2:
Create a Google service account. For instructions, see https://support.google.com/a/answer/7378726?hl=en

## Step 3:
Create a key in JSON format for this service account, then download it to your computer (as a .json file) and store it in a safe location. See https://cloud.google.com/iam/docs/creating-managing-service-account-keys 

## Step 4:
Grant this service account Editor access to your Google Slides presentation.

(If you don't have a presentation in place yet, go ahead and create one, then add as many blank slides to the presentation as you'll have images. You'll need the IDs corresponding to these blank slides in order to get the code to work.)

You can do so by clicking the 'Share' button within the presentation and then entering the service account's email within the box that appears. The address of this service account can be found within the 'Service account details' page of your service account within the Google Cloud platform.

## Step 5:
Set up a Cloud Storage bucket within your Google Cloud Platform project. The following link should help, even though it applies to a different project: https://support.google.com/chrome/a/answer/10391616?hl=en

As with the Google Cloud Platform as a whole, you may incur expenses when using Google Cloud Storage.

## Step 6:
Grant your service account access to this bucket. Click on the name of the bucket within the 'Browser' tab in the Cloud Storage menu, then go to the 'Permissions' tab. Click 'ADD' and then enter your service account email within the 'New Members' box. Next, enter roles for this user. I used the following two roles:

Storage Admin

Storage Object Creator

However, you may prefer to choose less restrictive roles depending on your needs.

## Step 7:
Enable the Google Slides API for your project. To do so, enter 'Slides API' within the search box near the top of the Google Cloud Platform window. Click on the 'Google Slides API' result and then select the blue 'Enable' button. 

For other projects, you may want to later enable other APIs as well, such as the Google Drive and Google Sheets APIs. However, those won't be necessary for this particular program.






In [1]:
import upload_image_and_add_to_slide_v2 # For more information about this Python script, please see the comments within the .py file.

import time 
start_time = time.time() # Allows the program's runtime to be measured

First, I'll create a set of image name/Google Slide ID pairs. These pairs will be fed into the following for loops in order to upload each image to its corresponding slide.

The image names can be found in the image folders within this program. To find the Google Slide ID pairs, open each slide within a web browser, then go to the far right of the URL to locate the ID. 

For example, the URL for the first slide in my presentation that will contain an image is https://docs.google.com/presentation/d/1xJfItB6w7hH0Nq2va-B1nUJFKGto_sFKxdMRvrMRvsI/edit#slide=id.geb3a1584f7_0_138 . The ID follows the 'id." in the URL, so in this case it's geb3a1584f7_0_138. 

In [2]:
score_graphs = [] # This list will store lists in which the first element is the image name and the second element is the slide ID. The benefit of using a list is that image-slide pairs are kept together. (Folder paths will be specified within the for loops, so they don't need to be added here.)

# Note that file names can't have spaces, or else Google will reject them. The following Google error message excerpt explains what names are permissible:
# "The object ID . . . should start with a word character [a-zA-Z0-9_] and then followed by any number of the following characters [a-zA-Z0-9_-:]"

score_graphs.append(["Total_score_changes","geb3a1584f7_0_138"]) # This line specifies that I want to add the 'Total_score_changes' image to the slide whose ID is geb3a1584f7_0_138. 
score_graphs.append(["Bayville_score_changes","geb3a1584f7_0_5"])
score_graphs.append(["Cardinal_score_changes","geb3a1584f7_0_120"])
score_graphs.append(["Central_score_changes","geb3a1584f7_0_123"])
score_graphs.append(["Eagle_score_changes","geb3a1584f7_0_126"])
score_graphs.append(["East_River_score_changes","geb3a1584f7_0_129"])
score_graphs.append(["Fair_Lake_score_changes","geb3a1584f7_0_132"])
score_graphs.append(["Olive_score_changes","geb3a1584f7_0_135"])
score_graphs.append(["Westwood_score_changes","geb3a1584f7_0_141"])

coeff_graphs = [] # I recommend creating different lists for graphs with different sizes. That's because the for loops below apply the same transformation to each set of slides. The images in score_graphs have a different size than the images in coeff_graphs, so to align each set of images correctly, I separated them into two lists.

coeff_graphs.append(["Total_coeffs","geb3a1584f7_0_165"])
coeff_graphs.append(["Bayville_coeffs","geb3a1584f7_0_144"])
coeff_graphs.append(["Cardinal_coeffs","geb3a1584f7_0_147"])
coeff_graphs.append(["Central_coeffs","geb3a1584f7_0_150"])
coeff_graphs.append(["Eagle_coeffs","geb3a1584f7_0_153"])
coeff_graphs.append(["East_River_coeffs","geb3a1584f7_0_156"])
coeff_graphs.append(["Fair_Lake_coeffs","geb3a1584f7_0_159"])
coeff_graphs.append(["Olive_coeffs","geb3a1584f7_0_162"])
coeff_graphs.append(["Westwood_coeffs","geb3a1584f7_0_168"])


# coeff_graphs.append(["",""]) # Template for adding additional items

# score_graphs.append(["",""])

Next, I'll import my Google service account path into the code. This service account path will be used for the following for loops.

In [3]:
with open('..\\keys\\public_service_account_key_path.txt') as fin:
    api_key_path = fin.readline()

# Alternately, you can replace the above two lines with: api_key_path = 'insert_your_key_path_here.' However, I chose to store the path as a file in a different folder.

Next, I'll use a for loop to iterate through the file/slide ID pairs for the graphs in the score_graphs folder.

In [4]:
chart_set = score_graphs

# The full path to the presentation is https://docs.google.com/presentation/d/1xJfItB6w7hH0Nq2va-B1nUJFKGto_sFKxdMRvrMRvsI/ . The last part of that URL (1xJfItB6w7hH0Nq2va-B1nUJFKGto_sFKxdMRvrMRvsI), excluding the forward slash, is the presentation's ID.

for i in range(len(chart_set)): 
    print("Now on image number", i, "(starting from 0)")
    upload_image_and_add_to_slide_v2.upload_image_and_add_to_slide(image_folder_path = 'score_graphs\\', image_file_name = chart_set[i][0], image_file_extension = '.png', service_account_path = api_key_path, scopes = ['https://www.googleapis.com/auth/devstorage.read_write', 'https://www.googleapis.com/auth/drive'], presentation_id = '1xJfItB6w7hH0Nq2va-B1nUJFKGto_sFKxdMRvrMRvsI', page_object_id = chart_set[i][1], scaleX = 2.0, scaleY = 2.0, translateX = 501250, translateY = -1350000, bucket_name = 'public_bucket_for_slides_import')

    # For more information on this function, see its definition within upload_image_and_add_to_slide_v2.

    # The output below provides updates on the progression of the function. Specifically, it denotes when a file has been uploaded to the cloud; when a URL was generated; when the pre-existing copy of that file, if any, was deleted from the slide; and when the image was deleted from the Google Cloud Storage bucket. It also notes how long the image was accessible. 



Now on image number 0 (starting from 0)
File uploaded to cloud.
Signed URL generated.
Image already present on slide. Request added to delete the pre-existing copy so that a new one can be added.
Image added to slide. Pre-existing image deleted if requested.
Blob Total_score_changes deleted.
Image was accessible for 2.7511 second(s).
Now on image number 1 (starting from 0)
File uploaded to cloud.
Signed URL generated.
Image already present on slide. Request added to delete the pre-existing copy so that a new one can be added.
Image added to slide. Pre-existing image deleted if requested.
Blob Bayville_score_changes deleted.
Image was accessible for 2.2233 second(s).
Now on image number 2 (starting from 0)
File uploaded to cloud.
Signed URL generated.
Image already present on slide. Request added to delete the pre-existing copy so that a new one can be added.
Image added to slide. Pre-existing image deleted if requested.
Blob Cardinal_score_changes deleted.
Image was accessible for 2.15

Next, I'll upload the images in the coeff_graph folder to the Google Slides presentation using a similar for loop. Note that the scale and translation variables are different, as these graphs have different dimensions than those in the score_graphs folder.

In [5]:
chart_set = coeff_graphs
for i in range(len(chart_set)): 
    print("Now on image number", i, "(starting from 0)")
    upload_image_and_add_to_slide_v2.upload_image_and_add_to_slide(image_folder_path = 'coeff_graphs\\', image_file_name = chart_set[i][0], image_file_extension = '.png', service_account_path = api_key_path, scopes = ['https://www.googleapis.com/auth/devstorage.read_write', 'https://www.googleapis.com/auth/drive'], presentation_id = '1xJfItB6w7hH0Nq2va-B1nUJFKGto_sFKxdMRvrMRvsI', page_object_id = chart_set[i][1], scaleX = 2.3, scaleY = 2.3, translateX = -100000, translateY = -2000000, bucket_name = 'public_bucket_for_slides_import')

Now on image number 0 (starting from 0)
File uploaded to cloud.
Signed URL generated.
Image already present on slide. Request added to delete the pre-existing copy so that a new one can be added.
Image added to slide. Pre-existing image deleted if requested.
Blob Total_coeffs deleted.
Image was accessible for 2.3314 second(s).
Now on image number 1 (starting from 0)
File uploaded to cloud.
Signed URL generated.
Image already present on slide. Request added to delete the pre-existing copy so that a new one can be added.
Image added to slide. Pre-existing image deleted if requested.
Blob Bayville_coeffs deleted.
Image was accessible for 2.0786 second(s).
Now on image number 2 (starting from 0)
File uploaded to cloud.
Signed URL generated.
Image already present on slide. Request added to delete the pre-existing copy so that a new one can be added.
Image added to slide. Pre-existing image deleted if requested.
Blob Cardinal_coeffs deleted.
Image was accessible for 2.3077 second(s).
Now on 

The following code reports how long it took this script to run.

In [6]:
end_time = time.time()
run_time = end_time - start_time
run_minutes = run_time // 60
run_seconds = run_time % 60
print("Completed run at",time.ctime(end_time),"(local time)")
print("Total run time:",'{:.2f}'.format(run_time),"second(s) ("+str(run_minutes),"minute(s) and",'{:.2f}'.format(run_seconds),"second(s))") # Only meaningful when the program is run nonstop from start to finish


Completed run at Mon Aug 30 16:31:30 2021 (local time)
Total run time: 47.43 second(s) (0.0 minute(s) and 47.43 second(s))


The latest set of images in the coeff_graphs and score_graphs folders have now been updated within the Google Slides presentation. In the event that I need to update the images once again, I can simply copy the newest set into these folders (while keeping the file names the same). This saves lots of time compared to manually uploading each chart into Google Slides.

I hope this program proves useful for you as well!